# Preprocessing
Naive bayes will be the base model, and it will struggle with stop words and common words. For that reason, we need to preprocess the data.

preprocessing steps include
1. cleaning extra newline characters
2. remove accented characters to ASCII
3. Expand contractions
4. Lowercase(?) text
5. Convert numbers to words
6. Remove numbers
7. Remove stop words
8. Lemmatization

There are a variety of python packages that will help with these steps. They are

# Table of packages here

In [ ]:
# imports
import requests
import pandas as pd
import os

In [ ]:
os.chdir('..')
os.getcwd()


In [ ]:
# get data
from src.data.make_dataset import get_book
# from data.raw.book_urls import book_urls # not working?
book_urls = {'great_gatsby':'https://www.gutenberg.org/cache/epub/64317/pg64317.txt',
'the_sun_also_rises':'https://www.gutenberg.org/cache/epub/67138/pg67138.txt',
'a_tale_of_two_cities':'https://www.gutenberg.org/cache/epub/98/pg98.txt'}


In [ ]:
data_set = {}
for title, url in book_urls.items():
    print(url)
    data_set[title] = get_book(url)

# Bookends
Books in project gutenberg have lots of extra text at the end of the text file. Most of this is legalese and terms of use information and this is unneeded for the language modeling.

Let's look at some examples and then create a framework to remove it from the text

In [ ]:
great_gatsby = data_set['great_gatsby']

In [ ]:
great_gatsby[-9000:-5000]

In [ ]:
# end pattern
print('*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***')
end_gutenberg = '*** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***'

In [ ]:
pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
p = re.compile(pattern)
print('the span of the found text is ', re.search(pattern, great_gatsby).span())
print('the start of the found text is ', re.search(pattern, great_gatsby).start())
print('we should remove everything after the start fo the end of book pattern')
re.search(pattern, great_gatsby).start()

In [ ]:
# all books in project gutenberg end with 
# *** END OF THE PROJECT GUTENBERG EBOOK {Title} ***
# *** END OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***

import re
def remove_bookend(book:str)->str:
    """removes the extra end of the book in project gutenberg"""
    end_of_book_pattern = r'\*\*\* END OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
    match = re.search(end_of_book_pattern, book)
    if match is None:
        print('could not find project gutenberg ending')
        raise ValueError
    last_character = match.start()
    return book[:last_character]

### Unittest

In [ ]:
test_book_ending ='This is the end of the book. *** END OF THE PROJECT GUTENBERG EBOOK THE the book title with number 10 ***'
test_book_no_ending = remove_bookend(test_book_ending)
assert test_book_no_ending == 'This is the end of the book. '

# Remove book start
This is a harder problem. 
We do know that all project gutenberg books have boiler plate starting text, which we can find and remove. However, some books have table of contents, other books have introductions, preambles, etc.


In [ ]:
def remove_book_start(book:str)->str:
    """removes the boiler plate beginning part of the book in project gutenberg"""
    start_of_book_pattern = r'\*\*\* START OF THE PROJECT GUTENBERG EBOOK [\w\d\s]+ \*\*\*'
    match = re.search(start_of_book_pattern, book)
    if match is None:
        print('could not find project gutenberg beginning')
        raise ValueError
    first_character = match.end()
    return book[first_character:]

## Unittest

In [ ]:
test_book_start = '\ufeffThe Project Gutenberg eBook of The Great Gatsby        This ebook is for the use of anyone anywhere in the United States and  most other parts of the world at no cost and with almost no restrictions  whatsoever. You may copy it, give it away or re-use it under the terms  of the Project Gutenberg License included with this ebook or online  at www.gutenberg.org. If you are not located in the United States,  you will have to check the laws of the country where you are located  before using this eBook.    Title: The Great Gatsby      Author: F. Scott Fitzgerald    Release date: January 17, 2021 [eBook #64317]    Language: English        *** START OF THE PROJECT GUTENBERG EBOOK THE GREAT GATSBY ***          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear the go'
test_book_no_start = remove_book_start(test_book_start)
assert test_book_no_start == '          The Great Gatsby        by      F. Scott Fitzgerald                                 Table of Contents    I  II  III  IV  V  VI  VII  VIII  IX                                    Once again                                    to                                   Zelda      Then wear the go'

In [ ]:
gg = remove_book_start(great_gatsby)
gg = remove_bookend(gg)
gg[:1000]

## END
Left off here

Next steps

In [ ]:
# questions
# what are the most common words?
# what words are most commonly associated with an author


In [ ]:
# first, remove unwanted new line and tab characters from the text
for char in ["\n", "\r", "\d", "\t", "\s\s\s"]:
    gg = gg.replace(char, " ")

In [ ]:
# remove header, footer from gutenberg
gg[:1000].split()